In [2]:
!pip install --upgrade tensorflow keras


In [3]:
from keras.applications.resnet_v2 import ResNet50V2 , preprocess_input as resnet_preprocess
from keras.applications.densenet import DenseNet121, preprocess_input as densenet_preprocess
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Input, Lambda
input_shape = (331,331,3)
input_layer = Input(shape=input_shape)#first feature extractor
preprocessor_resnet = Lambda(resnet_preprocess)(input_layer)
resnet50v2 = ResNet50V2(weights = 'imagenet',include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_resnet)
preprocessor_densenet = Lambda(densenet_preprocess)(input_layer)
densenet = DenseNet121(weights = 'imagenet',include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_densenet)
merge = concatenate([resnet50v2,densenet])
stacked_model = Model(inputs = input_layer, outputs = merge)
stacked_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 331, 331,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 331, 331,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 331, 331,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50v2          │ (None, 2048)      │ 23,564,800 │ lambda[0][0]      │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ densenet121         │ (None, 1024)      │  7,037,504 │ lambda_1[0][0]    │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3072)      │          0 │ resnet50v2[0][0], │
│ (Concatenate)       │                   │            │ densenet121[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 30,602,304 (116.74 MB)

 Trainable params: 30,473,216 (116.25 MB)

 Non-trainable params: 129,088 (504.25 KB)

In [18]:
import os
import shutil
import pandas as pd
train_dir = r"D:\ai ml\ai2\train\\"
labels_dataframe = pd.read_csv(r"D:\ai ml\ai2\labels.csv")
dog_breeds = sorted(list(set(labels_dataframe['breed'])))
n_classes = len(dog_breeds)
class_to_num = dict(zip(dog_breeds, range(n_classes)))
labels_dataframe['file_path'] = labels_dataframe['id'].apply(lambda x:train_dir+f"{x}.jpg")
labels_dataframe['breed'] = labels_dataframe.breed.map(class_to_num)

In [20]:
from keras.utils import to_categorical
y_train = to_categorical(labels_dataframe.breed)

In [22]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tqdm import tqdm

def feature_extractor(df):
    img_size = (331, 331, 3)
    data_size = len(df)
    batch_size = 20
    X = np.zeros([data_size, 3072], dtype=np.uint8)

    
    datagen = ImageDataGenerator()

    generator = datagen.flow_from_dataframe(df,
        x_col='file_path', class_mode=None,batch_size=20, shuffle=False, target_size=(img_size[:2]), color_mode='rgb')
    i = 0

    for input_batch in tqdm(generator):
        input_batch = stacked_model.predict(input_batch)
        X[i * batch_size: (i + 1) * batch_size] = input_batch
        i += 1
        if i * batch_size >= data_size:
            break
    return X

X_train = feature_extractor(labels_dataframe)


Found 10222 validated image filenames.


  0%|          | 0/512 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


  0%|          | 1/512 [00:10<1:26:32, 10.16s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


...

100%|█████████▉| 510/512 [41:36<00:09,  4.61s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


100%|█████████▉| 511/512 [41:40<00:04,  4.60s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


100%|█████████▉| 511/512 [41:45<00:04,  4.90s/it]


In [23]:
import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout
predictor_model = keras.models.Sequential([
    InputLayer(X_train.shape[1:]),
    Dropout(0.7),
    Dense(n_classes, activation='softmax')
])
predictor_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
#Prepare call backs
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/checkpoint.keras',
                             monitor = 'val_loss',mode = 'min',save_best_only= True)
lr = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.5,patience = 3,min_lr = 0.00001)
my_callback=[EarlyStop_callback,checkpoint]

In [26]:
#Train simple DNN on extracted features.# here X is bottleneck feature extracted by using stacked pretrained model 
history_graph = predictor_model.fit(X_train , y_train,
            batch_size=128,
            epochs=60,
            validation_split=0.1 ,
           callbacks = my_callback)

Epoch 1/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1238 - loss: 4.2066 - val_accuracy: 0.7703 - val_loss: 1.5026
Epoch 2/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6632 - loss: 1.5044 - val_accuracy: 0.8319 - val_loss: 0.8716
Epoch 3/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7792 - loss: 0.9420 - val_accuracy: 0.8426 - val_loss: 0.6848
Epoch 4/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8349 - loss: 0.7110 - val_accuracy: 0.8543 - val_loss: 0.5996
Epoch 5/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8606 - loss: 0.5787 - val_accuracy: 0.8573 - val_loss: 0.5520
Epoch 6/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8836 - loss: 0.4859 - val_accuracy: 0.8514 - val_loss: 0.5226
Epoch 7/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8949 - loss: 0.4281 - val_accuracy: 0.8622 - val_loss: 0.5004
Epoch 8/60
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9053 - loss: 0.3912 - val_accuracy: 0.8631 - val_loss:

In [28]:
predictor_model.export('D:\\ai ml\\ai2\\predict')
stacked_model.export('D:\\ai ml\\ai2\\feature')

INFO:tensorflow:Assets written to: D:\ai ml\ai2\predict\assets


INFO:tensorflow:Assets written to: D:\ai ml\ai2\predict\assets


Saved artifact at 'D:\ai ml\ai2\predict'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3072), dtype=tf.float32, name='keras_tensor_629')
Output Type:
  TensorSpec(shape=(None, 120), dtype=tf.float32, name=None)
Captures:
  2321646453392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2321646456272: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: D:\ai ml\ai2\feature\assets


INFO:tensorflow:Assets written to: D:\ai ml\ai2\feature\assets


Saved artifact at 'D:\ai ml\ai2\feature'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 331, 331, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3072), dtype=tf.float32, name=None)
Captures:
  2323383881552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383880208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383883088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383882128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383880400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383880016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383884624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383885776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383885968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383885584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2323383885008: TensorSpec(s

In [37]:
from keras.preprocessing.image import load_img, img_to_array
img = load_img("C:\\Users\\Madhav Madan\\Downloads\\lebra.jpg", target_size=(331,331))
img = img_to_array(img)
img = np.expand_dims(img,axis = 0) # this is creating tensor(4Dimension)
    
extracted_features = stacked_model.predict(img)
y_pred = predictor_model.predict(extracted_features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [39]:
def get_key(val): 
        for key, value in class_to_num.items(): 
         if val == value: 
             return key 
            
pred_codes = np.argmax(y_pred, axis = 1)
predicted_dog_breed = get_key(pred_codes)


In [41]:
print(predicted_dog_breed)

labrador_retriever


In [43]:
X_train.shape[1:]

(3072,)

In [45]:
predictor_model.save_weights("D:\\ai ml\\ai2\\predict.weights.h5")

In [51]:
stacked_model.save_weights("D:\\ai ml\\ai2\\extd.weights.h5")